## __Build Database for Data Scientist Job Popularity Analysis__

Owners: Alexa Aguirre / Bernardo Arambula / Yumi Jin

In [145]:
# Import packages
import os
import re
import time
import random
import warnings
import requests
import mysql.connector
from bs4 import BeautifulSoup

### __Part 1 - Scraping and Saving HTML Content__
Since LinkedIn has __strict antiscraping policies__ and __a stable layout__, we prefer to use Beautiful Soup and Requests instead of Selenium to extract the data we need.  

First we need to saved the job listings url to a list for further usage.

In [2]:
# Initialize session and set headers
session = requests.Session()
headers = {
    'User-agent': 'Mozilla/5.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive'
}

# Set base URL and query parameters
base_url = "https://www.linkedin.com/jobs/search/"
query_params = {
    'distance': '25',
    'geoId': '103644278',
    'keywords': 'data scientist',
    'origin': 'JOBS_HOME_KEYWORD_HISTORY',
    'refresh': 'true',
    'start': 0
}

In [3]:
# Create a empty list to store the jobs' url
links = []

for page in range(1, 120): 
    # Update the start value in the query parameter
    query_params['start'] = (page - 1) * 25
    try:
        response = session.get(base_url, params=query_params, headers=headers)
        # Random sleep to avoid being detected as a bot
        time.sleep(random.uniform(10, 16)) 
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the list containing job results
        jobs_list = soup.find('ul', class_='jobs-search__results-list')
        if jobs_list:
            # Extract all job cards from the list
            job_cards = jobs_list.find_all('li', recursive=False)  # 'recursive=False' ensures it only looks at direct children
            for card in job_cards:
                link_tag = card.find('a', class_='base-card__full-link')
                if link_tag and link_tag.get('href'):
                    links.append(link_tag.get('href'))
                    
    except requests.exceptions.RequestException as e:
        print(e)
    except Exception as e:
        print(f"An error occurred: {e}")     
# print(links) 

HTTPSConnectionPool(host='www.linkedin.com', port=443): Max retries exceeded with url: /jobs/search/?distance=25&geoId=103644278&keywords=data+scientist&origin=JOBS_HOME_KEYWORD_HISTORY&refresh=true&start=300 (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))
['https://www.linkedin.com/jobs/view/data-scientist-at-rue-gilt-groupe-3858021032?position=1&pageNum=0&refId=Q1hjp0FvPXg%2FDcg5CwvnkA%3D%3D&trackingId=m4vBNl%2BXpG1foOsJgbn%2BIw%3D%3D&trk=public_jobs_jserp-result_search-card', 'https://www.linkedin.com/jobs/view/data-scientist-at-rent-the-runway-3857251816?position=2&pageNum=0&refId=Q1hjp0FvPXg%2FDcg5CwvnkA%3D%3D&trackingId=VwQd0UGMwotxNUpmxvSYyg%3D%3D&trk=public_jobs_jserp-result_search-card', 'https://www.linkedin.com/jobs/view/data-scientist-at-rue-gilt-groupe-3863111235?position=3&pageNum=0&refId=Q1hjp0FvPXg%2FDcg5CwvnkA%3D%3D&trackingId=9bv%2BGHypCs15T5WBFHlVxw%3D%3D&trk=public_jobs_jserp-result_search-card'

In [4]:
len(links)

1560

We scraped __1,560__ data scientist job listings.  
However, many of them are __duplicated__ because a single job may be listed multiple times. And also because of the __antiscraping policies__, it's hard to parse all of them, so the actual number may __decrease__ once we parse and individually save them to HTML files.   

To avoid being blocked, we initially divide the list and then request and save each listing individually.

In [225]:
# Slice up the links to perform web-scraping to avoid being blocked
links1 = links[0:200]
links2 = links[200:400]
links3 = links[400:600]
links4 = links[600:800]
links5 = links[800:1000]
links6 = links[1000:1200]
links7 = links[1200:1400]
links8 = links[1400:1560]

In [227]:
for link in links8:
    # Random sleep to avoid being detected as a bot
    time.sleep(random.uniform(5, 8)) 
    
    # Extract the job ID using regular expression
    job_id = re.search(r'jobs/view/.+-([0-9]+)\?', link)
    job_id = job_id.group(1)
    
    # Define the filename using the job ID
    filename = f"{job_id}.html"
    
    # For each of the listing URLs, use `requests` to fetch the listing page
    page = requests.get(link, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Save each HTML content to a separate file on disk
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(soup.prettify())
    print(f"Successfully saved {filename}")

Successfully saved 3860178110.html
Successfully saved 3854663368.html
Successfully saved 3853735090.html
Successfully saved 3856411900.html
Successfully saved 3862979601.html
Successfully saved 3853535578.html
Successfully saved 3855850785.html
Successfully saved 3858005565.html
Successfully saved 3862707722.html
Successfully saved 3860177195.html
Successfully saved 3859408313.html
Successfully saved 3851331090.html
Successfully saved 3858583560.html
Successfully saved 3857211517.html
Successfully saved 3860582081.html
Successfully saved 3862940904.html
Successfully saved 3862312442.html
Successfully saved 3862308771.html
Successfully saved 3855866045.html
Successfully saved 3860723838.html
Successfully saved 3858976610.html
Successfully saved 3855858013.html
Successfully saved 3858176868.html
Successfully saved 3860581193.html
Successfully saved 3860726934.html
Successfully saved 3860174658.html
Successfully saved 3848276079.html
Successfully saved 3858224052.html
Successfully saved 3

---

### __Part 2 - Get the needed information from the HTML files__
Now, we loop through each HTML files we just saved, and extract the needed information.

In [234]:
# Set the relative path
directory = '/Users/yumi/ucdavis/Winter Quarter/BAX-422/Final Project/html files/'

# create an empty list to store the job_info for future usage
job_list = []

# Loop through each saved HTML file from the path 
for filename in os.listdir(directory):
    # Check if the file ends with .html
    if filename.endswith(".html"):
        # Construct the full file path using the relative directory
        filepath = os.path.join(directory, filename)
        # Read the file content
        with open(filepath, 'r', encoding='utf-8') as file:
            html_content = file.read()
            # For each HTML file, use `BeautifulSoup` to parse the file content
            soup = BeautifulSoup(html_content, 'html.parser')
            
            job_id = filename.split(".")[0]
            job_title = soup.find('h1', class_='top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title')
            if job_title:
                job_title = job_title.get_text(strip=True)
            company_name = soup.find('a', class_='topcard__org-name-link topcard__flavor--black-link').get_text(strip=True)
            location = soup.find('span', class_='topcard__flavor topcard__flavor--bullet').get_text(strip=True)
            post_date = soup.find('span', class_='posted-time-ago__text').get_text(strip=True)
            applicant_num = soup.find('span', class_='num-applicants__caption')
            if applicant_num:
                applicant_num = applicant_num.get_text(strip=True)
            else:
                applicant_num = soup.find('figcaption', class_='num-applicants__caption')
                if applicant_num:
                    applicant_num = applicant_num.get_text(strip=True)
                else:
                    applicant_num = applicant_num.get_text(strip=True)
            salary = soup.find('div', class_='salary compensation__salary')
            if salary:
                salary = salary.get_text(strip=True)
            else:
                salary = soup.find('div', class_='salary compensation__salary')
            seniority_level = soup.select('span.description__job-criteria-text.description__job-criteria-text--criteria')[0].get_text(strip=True)
            employment_type = soup.select('span.description__job-criteria-text.description__job-criteria-text--criteria')[1].get_text(strip=True)
            job_function = soup.select('span.description__job-criteria-text.description__job-criteria-text--criteria')[2].get_text(strip=True)
            industries = soup.select('span.description__job-criteria-text.description__job-criteria-text--criteria')[3].get_text(strip=True)

            # Extract the required information
            job_data = {
                'job_id': job_id,
                'job_title': job_title,
                'company_name': company_name,
                'location': location,
                'post_date': post_date,
                'applicant_num': applicant_num,
                'salary': salary,
                'seniority_level': seniority_level,
                'employment_type': employment_type,
                'job_function': job_function,
                'industries': industries
            }
            
        # Store the job_data to the list for future usage
        job_list.append(job_data)
        
        # Print the extracted data to screen
        for key, value in job_data.items():
            print(f"{key}: {value}")
        print("\n---\n")

job_id: 3855866045
job_title: Senior Data Scientist - Digital Identity
company_name: Walmart
location: San Bruno, CA
post_date: 4 days ago
applicant_num: 162 applicants
salary: None
seniority_level: Mid-Senior level
employment_type: Full-time
job_function: Analyst, Information Technology, and Advertising
industries: Retail, Advertising Services, and Data Infrastructure and Analytics

---

job_id: 3862979601
job_title: Data Scientist, 1+ Years of Experience
company_name: Snap Inc.
location: New York, United States
post_date: 1 day ago
applicant_num: Over 200 applicants
salary: None
seniority_level: Entry level
employment_type: Full-time
job_function: Engineering and Information Technology
industries: Software Development

---

job_id: 3854076940
job_title: Senior Data Scientist
company_name: Moody's Ratings
location: New York, NY
post_date: 1 week ago
applicant_num: Over 200 applicants
salary: None
seniority_level: Mid-Senior level
employment_type: Full-time
job_function: Engineering an

In [235]:
len(job_list)

265

As mentioned earlier, the number of listings decreased to 265.   
We will store these in the 'LinkedIn' database for further analysis on the popularity.

---

### __Part 3 - Save the extracted data to MySQL database__  
Lastly, we import the extracted data into a MySQL database.   
We use 'job_id' as the primary key because it is a unique identifier.

In [240]:
# Ignore warnings
warnings.filterwarnings("ignore")
SQL_DB = "LinkedIn"

def create_sql_table(SQL_TABLE_CONTRIBUTOR, SQL_TABLE_CONTRIBUTOR_DEF):
    try:
        # Connect to server
        conn = mysql.connector.connect(host = 'localhost',
                                       user = 'root',
                                       password = 'E6j8xs8q.')
        cursor = conn.cursor()
        
        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE_CONTRIBUTOR + " " + SQL_TABLE_CONTRIBUTOR_DEF + ";";
        print(query)
        cursor.execute(query);
        cursor.close()
        conn.close()
        return

    except IOError as e:
        print(e)

In [241]:
SQL_TABLE_CONTRIBUTOR = "data_scientist"
SQL_TABLE_CONTRIBUTOR_DEF = "(" + \
        "job_id VARCHAR(100) NOT NULL PRIMARY KEY" + \
        ",job_title VARCHAR(100)" + \
        ",company_name VARCHAR(100)" + \
        ",location VARCHAR(100)" + \
        ",post_date VARCHAR(50)" + \
        ",applicant_num VARCHAR(100)" + \
        ",salary VARCHAR(200)" + \
        ",seniority_level VARCHAR(100)" + \
        ",employment_type VARCHAR(50)" + \
        ",job_function VARCHAR(200)" + \
        ",industries VARCHAR(200)" + \
        ")"
# Use the function to create database 
create_sql_table(SQL_TABLE_CONTRIBUTOR, SQL_TABLE_CONTRIBUTOR_DEF)

CREATE DATABASE IF NOT EXISTS LinkedIn
CREATE TABLE IF NOT EXISTS LinkedIn.data_scientist (job_id VARCHAR(100) NOT NULL PRIMARY KEY,job_title VARCHAR(100),company_name VARCHAR(100),location VARCHAR(100),post_date VARCHAR(50),applicant_num VARCHAR(100),salary VARCHAR(200),seniority_level VARCHAR(100),employment_type VARCHAR(50),job_function VARCHAR(200),industries VARCHAR(200));


In [242]:
try:
    # Connect to server
    conn = mysql.connector.connect(host = 'localhost',
                                   database = 'LinkedIn',
                                   user = 'root',
                                   password = 'E6j8xs8q.')

    cursor = conn.cursor()
    
    # Parameterized statement to match the table schema
    parameterized_stmt = ("INSERT INTO " + SQL_TABLE_CONTRIBUTOR +
                          " (job_id, job_title, company_name, location,"
                          "post_date, applicant_num, salary, seniority_level, "
                          "employment_type, job_function, industries) "
                          "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);")

    for data in job_list:
        # Explicitly specify the order of columns to match the INSERT statement
        values = tuple(data[column] for column in ['job_id', 'job_title', 'company_name', 'location', 'post_date',
                                                    'applicant_num', 'salary', 'seniority_level', 'employment_type',
                                                    'job_function', 'industries'])
        # Execute the insert statement with the values
        cursor.execute(parameterized_stmt, values)

        # Print the job_id of the inserted data
        print('Inserted:', data['job_id'])
    
    conn.commit()
    cursor.close()
    conn.close()
    
except IOError as e:
    print(e)

Inserted: 3855866045
Inserted: 3862979601
Inserted: 3854076940
Inserted: 3861595150
Inserted: 3850491140
Inserted: 3855858013
Inserted: 3862429639
Inserted: 3793462569
Inserted: 3853498180
Inserted: 3852769239
Inserted: 3849318319
Inserted: 3834720854
Inserted: 3849937328
Inserted: 3854764782
Inserted: 3863900526
Inserted: 3857557134
Inserted: 3824242038
Inserted: 3852719674
Inserted: 3859529661
Inserted: 3728296749
Inserted: 3846350605
Inserted: 3858182172
Inserted: 3853058784
Inserted: 3857518902
Inserted: 3849961616
Inserted: 3849103052
Inserted: 3803992164
Inserted: 3850523337
Inserted: 3834705716
Inserted: 3858582376
Inserted: 3855831106
Inserted: 3854663368
Inserted: 3860534046
Inserted: 3853578605
Inserted: 3860806643
Inserted: 3833821528
Inserted: 3853582288
Inserted: 3855850785
Inserted: 3860178110
Inserted: 3847659550
Inserted: 3849938087
Inserted: 3823550803
Inserted: 3806497192
Inserted: 3835755823
Inserted: 3823815300
Inserted: 3813591183
Inserted: 3818888713
Inserted: 385